In [ ]:
#import dependencies

import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
#from config import username, password
import psycopg2

from config import username
from config import password

In [ ]:
#read economic data data file

wv1 = pd.read_csv("Resources\WV1.csv",header=[0,3])
wv1.head()

In [ ]:
wv1.columns=wv1.columns.map("_".join)
wv1.head()

In [ ]:
#clean header names

wv1.columns = [x.lower().replace(" ","_").replace(",","") for x in wv1.columns]
                    
wv1.head()

In [ ]:
wv1_df=wv1.rename(columns ={"unnamed:_0_level_0_unnamed:_0_level_1":"country","unnamed:_7_level_0_2019":"per_capita_2019",\
                          "unnamed:_9_level_0_2019":"per_capita_growth_2019"})
wv1_df.head()

In [ ]:
wv1_df.dtypes

In [ ]:
#read poverty data file

wv2 = pd.read_csv("Resources\WV2.csv",header=[0,3])
wv2.head()

In [ ]:
#clean header names

wv2.columns=wv2.columns.map("_".join)
wv2.head()

In [ ]:
wv2.columns = [x.lower().replace(" ","_").replace(",","").replace("-","_") for x in wv2.columns]
                    
wv2.head()

In [ ]:
wv2.columns

In [ ]:
wv2_df=wv2.rename(columns ={"unnamed:_0_level_0_unnamed:_0_level_1":"country","unnamed:_10_level_0_2019":"contributing_family_workers_and_own_account_workers_female",\
                           'incidence_of_hiv_ages_15_49_(per_1000_uninfected_population_ages_15_49)_2020':'incidence_of_hiv_ages_15_49_per1000_2020'})
wv2_df.head()

In [ ]:
#read sustainability data file

wv3 = pd.read_csv("Resources\WV3.csv",header=[0,3])
wv3.head()

In [ ]:
#clean header names

wv3.columns=wv3.columns.map("_".join)
wv3.head()

In [ ]:
wv3.columns = [x.lower().replace(" ","_").replace("&","") for x in wv3.columns]
                    
wv3.head()

In [ ]:
wv3_df=wv3.rename(columns ={"unnamed:_0_level_0_unnamed:_0_level_1":"country", "ambient_pm2.5_air_pollution_2016":"ambient_pm2_5_air_pollution_2016"})
wv3_df.head()

In [ ]:
#drop empty rows

wv1_df.drop(wv1.index[227:400],inplace=True)
wv2_df.drop(wv2.index[226:400],inplace=True)
wv3_df.drop(wv3.index[226:400],inplace=True)

wv1_df

In [ ]:
#drop invalid cell data

wv1_df1=wv1_df.replace(to_replace="..", value="NaN")
wv2_df1=wv2_df.replace(to_replace="..", value="NaN")
wv3_df1=wv3_df.replace(to_replace="..", value="NaN")

wv1_df1

In [ ]:
wv1_df2=wv1_df1.replace(",", "", regex=True)
wv2_df2=wv2_df1.replace(",", "", regex=True)
wv3_df2=wv3_df1.replace(",", "", regex=True)

wv3_df2

In [ ]:
#create postgres connection

rds_connection_string = username + ":" + password + "@localhost:5432/world_development_indicators"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
engine.table_names()

In [ ]:
#write data to economy_size table

wv1_df2.to_sql(name='economy_size', con=engine, if_exists='append', index=False)

In [ ]:
#write data to ending_poverty table

wv2_df2.to_sql(name='ending_poverty', con=engine, if_exists='append', index=False)

In [ ]:
#write data to promoting_sustainability table

wv3_df2.to_sql(name='promoting_sustainability', con=engine, if_exists='append', index=False)